In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [6]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['0','1', '2', '3', '4', '5','6','7','8','9'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [8]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass


In [9]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])


In [12]:
np.array(sequences).shape

(300, 30, 126)

In [13]:
np.array(labels).shape

(300,)

In [14]:
X = np.array(sequences)

In [15]:
X.shape

(300, 30, 126)

In [16]:
y = to_categorical(labels).astype(int)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)


In [18]:
y_test.shape

(60, 10)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [20]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [21]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))


In [22]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [24]:
model.fit(X_train, y_train, epochs=2200, callbacks=[tb_callback])

Epoch 1/2200
8/8 [==============================] - 18s 853ms/step - loss: 2.3061 - categorical_accuracy: 0.0875
Epoch 2/2200
8/8 [==============================] - 1s 78ms/step - loss: 2.3011 - categorical_accuracy: 0.1333
Epoch 3/2200
8/8 [==============================] - 1s 97ms/step - loss: 2.3036 - categorical_accuracy: 0.1458
Epoch 4/2200
8/8 [==============================] - 1s 75ms/step - loss: 2.2909 - categorical_accuracy: 0.1583
Epoch 5/2200
8/8 [==============================] - 1s 77ms/step - loss: 2.2925 - categorical_accuracy: 0.1417
Epoch 6/2200
8/8 [==============================] - 1s 85ms/step - loss: 2.2838 - categorical_accuracy: 0.1458
Epoch 7/2200
8/8 [==============================] - 1s 91ms/step - loss: 2.2847 - categorical_accuracy: 0.1792
Epoch 8/2200
8/8 [==============================] - 1s 90ms/step - loss: 2.2300 - categorical_accuracy: 0.2250
Epoch 9/2200
8/8 [==============================] - 1s 83ms/step - loss: 2.2688 - categorical_accuracy: 0.1667

8/8 [==============================] - 1s 85ms/step - loss: 0.0654 - categorical_accuracy: 0.9708
Epoch 147/2200
8/8 [==============================] - 1s 84ms/step - loss: 0.0345 - categorical_accuracy: 0.9917
Epoch 148/2200
8/8 [==============================] - 1s 88ms/step - loss: 0.0248 - categorical_accuracy: 0.9875
Epoch 149/2200
8/8 [==============================] - 1s 83ms/step - loss: 0.0089 - categorical_accuracy: 1.0000
Epoch 150/2200
8/8 [==============================] - 1s 86ms/step - loss: 0.0170 - categorical_accuracy: 0.9958
Epoch 151/2200
8/8 [==============================] - 1s 95ms/step - loss: 0.0085 - categorical_accuracy: 0.9958
Epoch 152/2200
8/8 [==============================] - 1s 85ms/step - loss: 0.0159 - categorical_accuracy: 0.9917
Epoch 153/2200
8/8 [==============================] - 1s 89ms/step - loss: 0.0344 - categorical_accuracy: 0.9833
Epoch 154/2200
8/8 [==============================] - 1s 92ms/step - loss: 0.0834 - categorical_accuracy: 0.966

8/8 [==============================] - 1s 80ms/step - loss: 0.2399 - categorical_accuracy: 0.9125
Epoch 290/2200
8/8 [==============================] - 1s 60ms/step - loss: 0.1880 - categorical_accuracy: 0.9292
Epoch 291/2200
8/8 [==============================] - 1s 79ms/step - loss: 0.1591 - categorical_accuracy: 0.9500
Epoch 292/2200
8/8 [==============================] - 1s 84ms/step - loss: 0.1185 - categorical_accuracy: 0.9750
Epoch 293/2200
8/8 [==============================] - 1s 87ms/step - loss: 0.1115 - categorical_accuracy: 0.9708
Epoch 294/2200
8/8 [==============================] - 1s 76ms/step - loss: 0.0849 - categorical_accuracy: 0.9750
Epoch 295/2200
8/8 [==============================] - 1s 82ms/step - loss: 0.0588 - categorical_accuracy: 0.9875
Epoch 296/2200
8/8 [==============================] - 1s 81ms/step - loss: 0.0458 - categorical_accuracy: 0.9917
Epoch 297/2200
8/8 [==============================] - 1s 72ms/step - loss: 0.0378 - categorical_accuracy: 0.991

Epoch 432/2200
8/8 [==============================] - 1s 91ms/step - loss: 2.7885e-04 - categorical_accuracy: 1.0000
Epoch 433/2200
8/8 [==============================] - 1s 98ms/step - loss: 2.7446e-04 - categorical_accuracy: 1.0000
Epoch 434/2200
8/8 [==============================] - 1s 89ms/step - loss: 2.6851e-04 - categorical_accuracy: 1.0000
Epoch 435/2200
8/8 [==============================] - 1s 83ms/step - loss: 2.6406e-04 - categorical_accuracy: 1.0000
Epoch 436/2200
8/8 [==============================] - 1s 91ms/step - loss: 2.5901e-04 - categorical_accuracy: 1.0000
Epoch 437/2200
8/8 [==============================] - 1s 86ms/step - loss: 2.5393e-04 - categorical_accuracy: 1.0000
Epoch 438/2200
8/8 [==============================] - 1s 91ms/step - loss: 2.5143e-04 - categorical_accuracy: 1.0000
Epoch 439/2200
8/8 [==============================] - 1s 93ms/step - loss: 2.4515e-04 - categorical_accuracy: 1.0000
Epoch 440/2200
8/8 [==============================] - 1s 67ms/st

Epoch 502/2200
8/8 [==============================] - 1s 91ms/step - loss: 9.4437e-05 - categorical_accuracy: 1.0000
Epoch 503/2200
8/8 [==============================] - 1s 91ms/step - loss: 9.3244e-05 - categorical_accuracy: 1.0000
Epoch 504/2200
8/8 [==============================] - 2s 269ms/step - loss: 9.2076e-05 - categorical_accuracy: 1.0000
Epoch 505/2200
8/8 [==============================] - 2s 274ms/step - loss: 9.1045e-05 - categorical_accuracy: 1.0000
Epoch 506/2200
8/8 [==============================] - 2s 269ms/step - loss: 8.9832e-05 - categorical_accuracy: 1.0000
Epoch 507/2200
8/8 [==============================] - 2s 247ms/step - loss: 8.8863e-05 - categorical_accuracy: 1.0000
Epoch 508/2200
8/8 [==============================] - 2s 264ms/step - loss: 8.7732e-05 - categorical_accuracy: 1.0000
Epoch 509/2200
8/8 [==============================] - 2s 303ms/step - loss: 8.6341e-05 - categorical_accuracy: 1.0000
Epoch 510/2200
8/8 [==============================] - 3s 3

...

8/8 [==============================] - 1s 180ms/step - loss: 2.8428e-05 - categorical_accuracy: 1.0000
Epoch 1607/2200
8/8 [==============================] - 2s 202ms/step - loss: 2.8188e-05 - categorical_accuracy: 1.0000
Epoch 1608/2200
8/8 [==============================] - 2s 188ms/step - loss: 2.7916e-05 - categorical_accuracy: 1.0000
Epoch 1609/2200
8/8 [==============================] - 2s 191ms/step - loss: 2.7740e-05 - categorical_accuracy: 1.0000
Epoch 1610/2200
8/8 [==============================] - 1s 151ms/step - loss: 2.7519e-05 - categorical_accuracy: 1.0000
Epoch 1611/2200
8/8 [==============================] - 2s 197ms/step - loss: 2.7375e-05 - categorical_accuracy: 1.0000
Epoch 1612/2200
8/8 [==============================] - 1s 178ms/step - loss: 2.7221e-05 - categorical_accuracy: 1.0000
Epoch 1613/2200
8/8 [==============================] - 1s 179ms/step - loss: 2.6947e-05 - categorical_accuracy: 1.0000
Epoch 1614/2200
8/8 [==============================] - 1s 153ms/

Epoch 1675/2200
8/8 [==============================] - 1s 149ms/step - loss: 1.7684e-05 - categorical_accuracy: 1.0000
Epoch 1676/2200
8/8 [==============================] - 1s 174ms/step - loss: 1.7648e-05 - categorical_accuracy: 1.0000
Epoch 1677/2200
8/8 [==============================] - 1s 148ms/step - loss: 1.7453e-05 - categorical_accuracy: 1.0000
Epoch 1678/2200
8/8 [==============================] - 1s 130ms/step - loss: 1.7330e-05 - categorical_accuracy: 1.0000
Epoch 1679/2200
8/8 [==============================] - 1s 143ms/step - loss: 1.7238e-05 - categorical_accuracy: 1.0000
Epoch 1680/2200
8/8 [==============================] - 1s 129ms/step - loss: 1.7127e-05 - categorical_accuracy: 1.0000
Epoch 1681/2200
8/8 [==============================] - 1s 147ms/step - loss: 1.7028e-05 - categorical_accuracy: 1.0000
Epoch 1682/2200
8/8 [==============================] - 1s 147ms/step - loss: 1.6937e-05 - categorical_accuracy: 1.0000
Epoch 1683/2200
8/8 [===========================

Epoch 1812/2200
8/8 [==============================] - 2s 241ms/step - loss: 7.6655e-06 - categorical_accuracy: 1.0000
Epoch 1813/2200
8/8 [==============================] - 2s 256ms/step - loss: 7.6252e-06 - categorical_accuracy: 1.0000
Epoch 1814/2200
8/8 [==============================] - 2s 256ms/step - loss: 7.5716e-06 - categorical_accuracy: 1.0000
Epoch 1815/2200
8/8 [==============================] - 2s 256ms/step - loss: 7.5259e-06 - categorical_accuracy: 1.0000
Epoch 1816/2200
8/8 [==============================] - 2s 246ms/step - loss: 7.4916e-06 - categorical_accuracy: 1.0000
Epoch 1817/2200
8/8 [==============================] - 2s 234ms/step - loss: 7.4489e-06 - categorical_accuracy: 1.0000
Epoch 1818/2200
8/8 [==============================] - 2s 241ms/step - loss: 7.4002e-06 - categorical_accuracy: 1.0000
Epoch 1819/2200
8/8 [==============================] - 2s 250ms/step - loss: 7.3590e-06 - categorical_accuracy: 1.0000
Epoch 1820/2200
8/8 [===========================

Epoch 1949/2200
8/8 [==============================] - 2s 191ms/step - loss: 3.5454e-06 - categorical_accuracy: 1.0000
Epoch 1950/2200
8/8 [==============================] - 2s 191ms/step - loss: 3.5236e-06 - categorical_accuracy: 1.0000
Epoch 1951/2200
8/8 [==============================] - 2s 197ms/step - loss: 3.5012e-06 - categorical_accuracy: 1.0000
Epoch 1952/2200
8/8 [==============================] - 2s 203ms/step - loss: 3.4878e-06 - categorical_accuracy: 1.0000
Epoch 1953/2200
8/8 [==============================] - 2s 199ms/step - loss: 3.4590e-06 - categorical_accuracy: 1.0000
Epoch 1954/2200
8/8 [==============================] - 2s 200ms/step - loss: 3.4446e-06 - categorical_accuracy: 1.0000
Epoch 1955/2200
8/8 [==============================] - 1s 186ms/step - loss: 3.4242e-06 - categorical_accuracy: 1.0000
Epoch 1956/2200
8/8 [==============================] - 2s 198ms/step - loss: 3.4054e-06 - categorical_accuracy: 1.0000
Epoch 1957/2200
8/8 [===========================

8/8 [==============================] - 1s 173ms/step - loss: 1.6545e-06 - categorical_accuracy: 1.0000
Epoch 2086/2200
8/8 [==============================] - 1s 181ms/step - loss: 1.6481e-06 - categorical_accuracy: 1.0000
Epoch 2087/2200
8/8 [==============================] - 1s 175ms/step - loss: 1.6391e-06 - categorical_accuracy: 1.0000
Epoch 2088/2200
8/8 [==============================] - 1s 167ms/step - loss: 1.6242e-06 - categorical_accuracy: 1.0000
Epoch 2089/2200
8/8 [==============================] - 2s 186ms/step - loss: 1.6128e-06 - categorical_accuracy: 1.0000
Epoch 2090/2200
8/8 [==============================] - 1s 167ms/step - loss: 1.6048e-06 - categorical_accuracy: 1.0000
Epoch 2091/2200
8/8 [==============================] - 1s 172ms/step - loss: 1.5979e-06 - categorical_accuracy: 1.0000
Epoch 2092/2200
8/8 [==============================] - 2s 251ms/step - loss: 1.5924e-06 - categorical_accuracy: 1.0000
Epoch 2093/2200
8/8 [==============================] - 2s 216ms/

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            48896     
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 203,690
Trainable params: 203,690
Non-trainable params: 0
__________________________________________________

In [24]:
res = model.predict(X_test)

In [30]:
actions[np.argmax(res[4])]

'8'

In [31]:
actions[np.argmax(y_test[4])]

'0'

In [1]:
model.save('action.h5')

NameError: name 'model' is not defined

In [32]:
model.load_weights('action.h5')

In [33]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [34]:
yhat = model.predict(X_test)

In [35]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [36]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[53,  0],
        [ 1,  6]],

       [[54,  0],
        [ 0,  6]],

       [[55,  1],
        [ 0,  4]],

       [[55,  0],
        [ 1,  4]],

       [[55,  0],
        [ 0,  5]],

       [[53,  1],
        [ 0,  6]],

       [[56,  0],
        [ 0,  4]],

       [[49,  0],
        [ 0, 11]],

       [[54,  0],
        [ 0,  6]],

       [[54,  0],
        [ 0,  6]]], dtype=int64)

In [37]:
accuracy_score(ytrue, yhat)

0.9666666666666667

In [38]:
from scipy import stats

In [39]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

In [43]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            sign = actions[np.argmax(res)]
            print(sign)
            cv2.putText(image, sign, (100,100), cv2.FONT_HERSHEY_SIMPLEX,4 , (0, 0, 255), 2, cv2.LINE_AA )
            
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>

5
<class 'mediapipe.python.solution_base.SolutionOutputs'>
5
<class 'mediapipe.python.solution_base.SolutionOutputs'>
5
<class 'mediapipe.python.solution_base.SolutionOutputs'>
5
<class 'mediapipe.python.solution_base.SolutionOutputs'>
5
<class 'mediapipe.python.solution_base.SolutionOutputs'>
5
<class 'mediapipe.python.solution_base.SolutionOutputs'>
5
<class 'mediapipe.python.solution_base.SolutionOutputs'>
5
<class 'mediapipe.python.solution_base.SolutionOutputs'>
5
<class 'mediapipe.python.solution_base.SolutionOutputs'>
5
<class 'mediapipe.python.solution_base.SolutionOutputs'>
5
<class 'mediapipe.python.solution_base.SolutionOutputs'>
5
<class 'mediapipe.python.solution_base.SolutionOutputs'>
5
<class 'mediapipe.python.solution_base.SolutionOutputs'>
5
<class 'mediapipe.python.solution_base.SolutionOutputs'>
5
<class 'mediapipe.python.solution_base.SolutionOutputs'>
5
<class 'mediapipe.python.solution_base.SolutionOutputs'>
5
<class 'mediapipe.python.solution_base.SolutionOutputs

<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>

<class 'mediapipe.python.solution_base.SolutionOutputs'>
8
<class 'mediapipe.python.solution_base.SolutionOutputs'>
8
<class 'mediapipe.python.solution_base.SolutionOutputs'>
8
<class 'mediapipe.python.solution_base.SolutionOutputs'>
8
<class 'mediapipe.python.solution_base.SolutionOutputs'>
8
<class 'mediapipe.python.solution_base.SolutionOutputs'>
8
<class 'mediapipe.python.solution_base.SolutionOutputs'>
8
<class 'mediapipe.python.solution_base.SolutionOutputs'>
8
<class 'mediapipe.python.solution_base.SolutionOutputs'>
8
<class 'mediapipe.python.solution_base.SolutionOutputs'>
8
<class 'mediapipe.python.solution_base.SolutionOutputs'>
8
<class 'mediapipe.python.solution_base.SolutionOutputs'>
8
<class 'mediapipe.python.solution_base.SolutionOutputs'>
8
<class 'mediapipe.python.solution_base.SolutionOutputs'>
8
<class 'mediapipe.python.solution_base.SolutionOutputs'>
8
<class 'mediapipe.python.solution_base.SolutionOutputs'>
8
<class 'mediapipe.python.solution_base.SolutionOutputs'>

<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>